The goal of this project is to:
* get data from the kaggle connectomics competition into a suitable form for processing by TensorFlow
* construct a network following the published deep conv net architecture. aim to reproduce the published AUC
* improve the power of the model with newer network architectures

In [1]:
# Imports
#
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc, precision_recall_curve, accuracy_score, roc_auc_score
%matplotlib inline

In [2]:
# I/O functions provided by competition 
# https://github.com/bisakha/Connectomics/
#
def iter_loadtxt(filename, delimiter=',', skiprows=0, dtype=float):
    def iter_func():
        with open(filename, 'r') as infile:
            for _ in range(skiprows):
                next(infile)
            for line in infile:
                line = line.rstrip().split(delimiter)
                for item in line:
                    yield dtype(item)
        iter_loadtxt.rowlength = len(line)

    data = np.fromiter(iter_func(), dtype=dtype)
    data = data.reshape((-1, iter_loadtxt.rowlength))
    return data

In [3]:
# I/O functions provided by competition 
# https://github.com/bisakha/Connectomics/
#
def loadnet(filename):
    l1 = np.loadtxt(filename,delimiter=",")    

    #% Keep only 0/1 weights, ignore blocked connections
    Matrix = [[0 for x in range(int(l1.max()))] for x in range(int(l1.max()))]

    for i in range(0,len(l1)-1):
        if l1[i][2] > 0:
            l1[i][2] = 1

            Matrix[int(l1[i][0])-1][int(l1[i][1])-1] = 1

        if l1[i][2] <= 0:
            l1[i][2] = 0

            Matrix[int(l1[i][0])-1][int(l1[i][1])-1] = 0

    return np.array(Matrix)

In [4]:
# Read training dataset
filename = '../Tensor/kaggle_connect_data/normal-1/fluorescence_normal-1.txt'
%time F = iter_loadtxt(filename)

Wall time: 1min 14s


In [5]:
# Read validation dataset
filename = '../Tensor/kaggle_connect_data/normal-2/fluorescence_normal-2.txt'
%time F_valid = iter_loadtxt(filename)

Wall time: 1min 14s


In [6]:
# Read training connectivity matrix
filename = '../Tensor/kaggle_connect_data/normal-1/network_normal-1.txt'
scores = loadnet(filename)

In [7]:
# Read validation connectivity matrix
filename = '../Tensor/kaggle_connect_data/normal-2/network_normal-2.txt'
scores_valid = loadnet(filename)

Essentially, we want to create an "image set" of all possible 2-permutations of the data, + the average activity of the network. This will form our first tensor.

But Romaszko also downsampled the data using an activity threshold -- we should do the same downsampling first

In [8]:
def roma_ds(fluor):
    """
    Takes a 2-D numpy array of fluorescence time series (in many cells) and returns a downsampled version, following the
        filtering method published by Romaszko (threshold summed time-diff global network activity at 0.02)

    inputs---
        fluor: numpy array of fluorescence time series. rows are cells, columns are time points / frames
    outputs---
        fluor_ds: downsampled numpy array of fluorescence time series.
    """
    thresh = 20
    fluordiff = np.diff(fluor, axis=1)
    totF = np.sum(fluordiff, axis=0)
    fluor = fluor[:,totF>thresh]
    
    return fluor

In [9]:
def dunn_ds(fluor):
    """
    Takes a 2-D numpy array of fluorescence time series (in many cells) and returns a 
    downsampled version by taking every 100th frame

    inputs---
        fluor: numpy array of fluorescence time series. rows are cells, columns are time points / frames
    outputs---
        fluor_ds: downsampled numpy array of fluorescence time series.
    """
    
    return np.diff(fluor[:,::10])

Now we can create the training tensor

In [10]:
def pairwise_prep(fluor, connect):
    """
    Takes a 2-D numpy array of fluorescence time series (in many cells) and returns a 4-D numpy array ready for tensorflow,
        with each "image" constructed from a pair of fluorescence traces + average network activity. 
        
    This tensor incorporates several features from the Romaszko setup.
        1) fragments of length 330 frames are extracted, at random start positions, from each fluorescence trace
        2) equal representation of connected and non-connected pairs
        3) 1.2 million total examples
        4) equal representation of any included pairs -- meaning no connected pair, if included, is represented any more than
            another
        
    While at it, builds a structure of one-hot vectors (in this case, a single binary label) 
        to signify "connected" or "not connected"
    
    inputs---
        fluor: 2-D numpy array of fluorescence time series. rows are cells, columns are time points / frames
        connect: 2-D numpy array connectivity matrix summarizing all possible pairwise connectivity.
    outputs---
        fluor_tf: 4-D pairwise numpy array ready for tensorflow
        label_tf: a 1-D numpy array labeling connectivity for each possible pair in the dataset
    """
    num_cells = fluor.shape[0]
    num_samples = 330
    
    raw_samples = fluor.shape[1]
    
    # In theory cells could be auto-connected
    num_images_target = 1.2e6
    
    # Find all connected pairs
    cons = np.where(connect==1)
    num_con = len(cons[0])
    num_con_reps = np.floor(num_images_target/2/num_con).astype('int')
    
    num_images = num_con_reps*num_con*2
    fluor_tf = np.empty((num_images, 3, num_samples, 1),dtype='float32')
    label_tf = np.zeros((num_images,2),dtype='float32')
    
    avg_F = np.mean(fluor,axis=0)
    
    # Add conncted pairs to tensor
    cnt = 0
    for i in range(num_con):
        for j in range(num_con_reps):
            startpos = np.random.randint(0,raw_samples-num_samples,1)[0]
            fluor_tf[cnt,:,:,0] = np.vstack((fluor[cons[0][i],startpos:startpos+num_samples], 
                                                   fluor[cons[1][i],startpos:startpos+num_samples], 
                                                   avg_F[startpos:startpos+num_samples]))
            label_tf[cnt,0] = 1
            cnt += 1
    
    # Find all non-connected pairs
    # There are typically too many non-connected pairs to have any repetitions in the training set
    noncons = np.where(connect==0)
    
    # Sample randomly from noncons without replacement
    noncons_samp = (np.random.choice(noncons[0],num_images/2,replace=False), 
                    np.random.choice(noncons[1],num_images/2,replace=False))
    
    # Create final data structure
    for i in range(num_images//2):
        startpos = np.random.randint(0,raw_samples-num_samples,1)[0]
        fluor_tf[cnt,:,:,0] = np.vstack((fluor[noncons_samp[0][i],startpos:startpos+num_samples], 
                                               fluor[noncons_samp[1][i],startpos:startpos+num_samples], 
                                               avg_F[startpos:startpos+num_samples]))
        label_tf[cnt,1] = 1
        cnt += 1
            
            
    return fluor_tf, label_tf

In [11]:
# Downsample fluorescence time series using a hard population activity threshold 
#
ds = roma_ds(F.T)
ds_valid = roma_ds(F_valid.T)
F = []

C:\Users\Tim\Anaconda3\lib\site-packages\ipykernel\__main__.py:14: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 1; dimension is 179500 but corresponding boolean dimension is 179499


In [12]:
# Standardize training data
#
vs = ds - np.mean(ds,axis=1)[:,None]
vs = vs/np.std(vs,axis=1)[:,None]
vs.shape

(1000, 1856)

In [13]:
# Standardize validation data
#
vs_valid = ds_valid - np.mean(ds_valid,axis=1)[:,None]
vs_valid = vs_valid/np.std(vs_valid,axis=1)[:,None]
vs_valid.shape

(1000, 1553)

In [14]:
# Prepare trainign data structure + labels
#
dtf, ltf = pairwise_prep(vs,scores)

C:\Users\Tim\Anaconda3\lib\site-packages\ipykernel\__main__.py:61: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
C:\Users\Tim\Anaconda3\lib\site-packages\ipykernel\__main__.py:62: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [15]:
dtf.shape

(1194900, 3, 330, 1)

In [16]:
ltf.shape

(1194900, 2)

Shuffle everything and save 4% as a validation set

In [17]:
dtf.shape

(1194900, 3, 330, 1)

OK, now we can set up our network layers using tensorflow

In [18]:
import tensorflow as tf
sess = tf.InteractiveSession()

In [19]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [20]:
# Standard 2-D convolution
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='VALID')

# Only used in the last layer
def max_pool_1x10(x):
    return tf.nn.max_pool(x, ksize=[1, 1, 10, 1],
                        strides=[1, 1, 10, 1], padding='VALID')

In [21]:
x = tf.placeholder(tf.float32, shape=[None,dtf.shape[1],dtf.shape[2],1])
y_ = tf.placeholder(tf.float32, shape=[None, 2])

First convolutional layer

In [22]:
W_conv1 = weight_variable([2, 5, 1, 18])
b_conv1 = bias_variable([18])

h_conv1 = tf.nn.tanh(conv2d(x, W_conv1) + b_conv1)

Second convolutional layer

In [23]:
W_conv2 = weight_variable([2, 5, 18, 40])
b_conv2 = bias_variable([40])

h_conv2 = tf.nn.relu(conv2d(h_conv1, W_conv2) + b_conv2)
h_pool2 = max_pool_1x10(h_conv2)

Third convolutional layer

In [24]:
W_conv3 = weight_variable([1, 1, 40, 15])
b_conv3 = bias_variable([15])

h_conv3 = tf.nn.relu(conv2d(h_pool2, W_conv3) + b_conv3)

Fully connected layer

In [25]:
W_fc1 = weight_variable([32 * 1 * 15, 100])
b_fc1 = bias_variable([100])

h_conv3_flat = tf.reshape(h_conv3, [-1, 32 * 1 * 15])
h_fc1 = tf.nn.tanh(tf.matmul(h_conv3_flat, W_fc1) + b_fc1)

Readout

In [26]:
W_fc2 = weight_variable([100, 2])
b_fc2 = bias_variable([2])

y_conv = tf.matmul(h_fc1, W_fc2) + b_fc2

In [43]:
def valid_eval(session, val_dat, val_lbl, N=14, fragLen=330):
    """
    Properly calidates current CNN filters by passing filters over retained validation set N number of times and averaging
    the set of predictiosn for each pair
    
    inputs---
        session: tensorflow session object
        val_dat: 2-D numpy array of downsampled fluorescence traces
        val_lbl: 2-D numpy array of connectivity labels for eah pair (connectivity matrix)
        N: number of separate start positions for each test fragment to be averaged for each pair
        fragLen: length of trained CNN filter, in time points/samples
    outputs---
        AUC for the validated predictions
    """
    avg_F = np.mean(val_dat,axis=0)

    startgap = np.ceil((val_dat.shape[1] - fragLen)/N).astype('int')
    true_lbl = np.zeros((val_dat.shape[0]*val_dat.shape[0],), dtype='float32')
    pred_lbl = np.zeros((val_dat.shape[0]*val_dat.shape[0],), dtype='float32')
    
    # Counter for the "true_lbl" array
    cnt_ = 0
    # Counter for the "pred_lbl" array
    cnt_u = 0
    for a in range(val_dat.shape[0]):
        if a%100 == 0:
            print('\r' + 'X'*(a//100))
        
        # Create batch array to send thru network
        im_eval = np.empty((N*val_dat.shape[0],3,fragLen,1), dtype='float32')
        
        # Count the number of traces in each batch
        cnt = 0
            
        for b in range(val_dat.shape[0]):
            
            for n in range(0, val_dat.shape[1] - fragLen, startgap):
                try:
                    im_eval[cnt,:,:,0] = np.vstack((val_dat[a,n:n+fragLen],
                                         val_dat[b,n:n+fragLen],
                                         avg_F[n:n+fragLen]))
                except:
                    from IPython.core.debugger import Tracer
                    Tracer()()
    
                cnt += 1
            
            # Keep track of the true labels
            if val_lbl[a,b] == 1:
                true_lbl[cnt_] = 1
            else:
                true_lbl[cnt_] = 0
            
            cnt_ += 1
        
        # Run batch through network
        pred_stop = sess.run(dispense_data, feed_dict={x: im_eval })[:,0]
        # Average output over each group of N traces
        for u in range(0, len(pred_stop), N):
            pred_lbl[cnt_u] = np.mean(pred_stop[u:u+N])
            cnt_u += 1        
    
    # Get ROC metrics
    fpr, tpr, thresholds = roc_curve(true_lbl, pred_lbl)
    
    return auc(fpr, tpr)
            


### Train and evaluate.

In theory we should be using a negative log-likelihood cost function (in order to match Romaszko exactly), but we'll go with a standard cross entropy here.

Also, Romaszko used a gradient descent optimizer with momentum -- we can change this later too

In [44]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y_conv, y_))
train_step = tf.train.AdamOptimizer(1e-3).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))

dispense_data = tf.nn.softmax(y_conv)

collect_prediction = tf.reduce_mean(y_conv,0)

accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
sess.run(tf.global_variables_initializer())

num_epochs = 10
num_batches = dtf.shape[0]/1000

for j in range(num_epochs):
    inds = np.random.choice(dtf.shape[0],replace=False,size=dtf.shape[0])
    dtf = dtf[inds,:,:,:]
    ltf = ltf[inds]
    score = 0
    loss = 0
    
    for i in range(0,dtf.shape[0],1000):

        
        im = dtf[i:i+1000,:,:,:]
        lbl = ltf[i:i+1000,:]#.astype('int')

        train_step.run(feed_dict={x: im, y_: lbl})
    
        fpr, tpr, thresholds = roc_curve(lbl[:,0], dispense_data.eval(feed_dict={x: im})[:,0])
        score += auc(fpr, tpr)
        
        loss += cross_entropy.eval(feed_dict={x: im, y_: lbl})
    
        if i%100000 == 0:
            print(i)
            

    print("step %d, training accuracy %g"%(j, score/num_batches))
    print("loss: {}".format(loss/num_batches))
    
    real = valid_eval(sess, vs_valid, scores_valid)
    print("Batch validation score: {}".format(real))


0
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
step 0, training accuracy 0.899445
loss: 0.3882453933511581

X
XX
XXX
XXXX
XXXXX
XXXXXX
XXXXXXX
XXXXXXXX
XXXXXXXXX
Batch validation score: 0.9209667420044925
0
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
step 1, training accuracy 0.915215
loss: 0.3615640078252185

X
XX
XXX
XXXX
XXXXX
XXXXXX
XXXXXXX
XXXXXXXX
XXXXXXXXX
Batch validation score: 0.9264413391322919
0
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
step 2, training accuracy 0.918206
loss: 0.3561718918926217

X
XX
XXX
XXXX
XXXXX
XXXXXX
XXXXXXX
XXXXXXXX
XXXXXXXXX
Batch validation score: 0.9291227168313412
0
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
step 3, training accuracy 0.919938
loss: 0.35272164453132465

X
XX
XXX
XXXX
XXXXX
XXXXXX
XXXXXXX
XXXXXXXX
XXXXXXXXX
Batch validation score: 0.930173425336805
0
100000
200000
300000
400000
500000
600000
